In [2]:
%load_ext IPython.extensions.autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../..')
from model import FinData
from model import train_valid_split
from model import CatboostFinModel

c:\Users\burko\OneDrive\Documents\GitHub\prices-predictions\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import datetime as dt

Let's start with basic Regularization

In [9]:
data = FinData("../../datasets/T_yandex_10min.csv")

data.insert_shifts_norms()
data.insert_rolling_means()
data.insert_exp_rolling_means()
data.insert_stochastic_oscillator()
data.insert_high_low_diff()
data.insert_holidays()
data.insert_rsi()
data.insert_bollinger()
data.insert_random_prediction()

data.restrict_time_down_stupidly(months=6)

X_train, X_val, y_train, y_val = train_valid_split(data=data.df, 
                                                   year=2024, month=12, day=5, 
                                                   numeric=data.numeric_features, cat=data.cat_features, target="direction_binary")

In [13]:
numeric = data.numeric_features

cat = data.cat_features

args = {"iterations" : 5000, 
        "depth" : 5, 
        "learning_rate" : 0.01, # тут слегка неадекватные параметры, которые можно менять 
        "use_best_model" : True, 
        # "l2_leaf_reg" : 200,
        "loss_function" : 'CrossEntropy', 
        "eval_metric" : 'Accuracy', 
        "cat_features" : cat, 
        "random_state" : 42,
        "early_stopping_rounds" : 5000}


model = CatboostFinModel(args)

model.set_datasets(X_train, X_val, y_train, y_val)
model.set_features(numeric, cat)

model.fit()

0:	learn: 0.5983831	test: 0.5909091	best: 0.5909091 (0)	total: 11ms	remaining: 55.1s
1:	learn: 0.5985315	test: 0.6010101	best: 0.6010101 (1)	total: 21.8ms	remaining: 54.6s
2:	learn: 0.5990507	test: 0.6060606	best: 0.6060606 (2)	total: 34.4ms	remaining: 57.3s
3:	learn: 0.5977156	test: 0.6161616	best: 0.6161616 (3)	total: 44ms	remaining: 55s
4:	learn: 0.5966773	test: 0.6212121	best: 0.6212121 (4)	total: 54.5ms	remaining: 54.4s
5:	learn: 0.5991248	test: 0.6060606	best: 0.6212121 (4)	total: 64.1ms	remaining: 53.3s
6:	learn: 0.6009048	test: 0.6161616	best: 0.6212121 (4)	total: 74.2ms	remaining: 52.9s
7:	learn: 0.6001632	test: 0.6060606	best: 0.6212121 (4)	total: 84.5ms	remaining: 52.7s
8:	learn: 0.5991248	test: 0.6212121	best: 0.6212121 (4)	total: 94.5ms	remaining: 52.4s
9:	learn: 0.5980865	test: 0.6262626	best: 0.6262626 (9)	total: 104ms	remaining: 51.7s
10:	learn: 0.5971223	test: 0.6161616	best: 0.6262626 (9)	total: 113ms	remaining: 51.3s
11:	learn: 0.5980865	test: 0.6111111	best: 0.62626

In [14]:
args2 = {**args, "reg_lambda":5} # Параметр регуляризации L2


model2 = CatboostFinModel(args2)

model2.set_datasets(X_train, X_val, y_train, y_val)
model2.set_features(numeric, cat)
model2.fit()

0:	learn: 0.5983831	test: 0.5909091	best: 0.5909091 (0)	total: 16ms	remaining: 1m 20s
1:	learn: 0.5985315	test: 0.6010101	best: 0.6010101 (1)	total: 31.7ms	remaining: 1m 19s
2:	learn: 0.5991248	test: 0.6060606	best: 0.6060606 (2)	total: 50.6ms	remaining: 1m 24s
3:	learn: 0.5980123	test: 0.5909091	best: 0.6060606 (2)	total: 66.5ms	remaining: 1m 23s
4:	learn: 0.5980123	test: 0.5959596	best: 0.6060606 (2)	total: 81.6ms	remaining: 1m 21s
5:	learn: 0.5983090	test: 0.6010101	best: 0.6060606 (2)	total: 98.8ms	remaining: 1m 22s
6:	learn: 0.5994215	test: 0.6111111	best: 0.6111111 (6)	total: 113ms	remaining: 1m 20s
7:	learn: 0.5997182	test: 0.6161616	best: 0.6161616 (7)	total: 126ms	remaining: 1m 18s
8:	learn: 0.5983831	test: 0.6060606	best: 0.6161616 (7)	total: 141ms	remaining: 1m 18s
9:	learn: 0.5982348	test: 0.6111111	best: 0.6161616 (7)	total: 155ms	remaining: 1m 17s
10:	learn: 0.5981606	test: 0.6111111	best: 0.6161616 (7)	total: 170ms	remaining: 1m 17s
11:	learn: 0.5981606	test: 0.6111111	b

In [16]:
args3 = {**args, "reg_lambda":0.1} # Параметр регуляризации L1


model3 = CatboostFinModel(args3)

model3.set_datasets(X_train, X_val, y_train, y_val)
model3.set_features(numeric, cat)
model3.fit()

0:	learn: 0.5983831	test: 0.5909091	best: 0.5909091 (0)	total: 12.9ms	remaining: 1m 4s
1:	learn: 0.5973448	test: 0.6060606	best: 0.6060606 (1)	total: 24.4ms	remaining: 1m
2:	learn: 0.5969740	test: 0.6111111	best: 0.6111111 (2)	total: 36.1ms	remaining: 1m
3:	learn: 0.5968998	test: 0.5959596	best: 0.6111111 (2)	total: 45.9ms	remaining: 57.3s
4:	learn: 0.5979381	test: 0.6060606	best: 0.6111111 (2)	total: 56.9ms	remaining: 56.8s
5:	learn: 0.5966773	test: 0.6161616	best: 0.6161616 (5)	total: 68ms	remaining: 56.6s
6:	learn: 0.5967515	test: 0.6212121	best: 0.6212121 (6)	total: 79.3ms	remaining: 56.6s
7:	learn: 0.5973448	test: 0.6212121	best: 0.6212121 (6)	total: 88ms	remaining: 54.9s
8:	learn: 0.5956390	test: 0.6212121	best: 0.6212121 (6)	total: 98.6ms	remaining: 54.7s
9:	learn: 0.5961581	test: 0.6313131	best: 0.6313131 (9)	total: 108ms	remaining: 53.8s
10:	learn: 0.5962323	test: 0.6313131	best: 0.6313131 (9)	total: 119ms	remaining: 54s
11:	learn: 0.5954906	test: 0.6111111	best: 0.6313131 (9)

In [20]:
args4 = {**args, "l2_leaf_reg": 200} # Параметр регуляризации L1


model4 = CatboostFinModel(args4)

model4.set_datasets(X_train, X_val, y_train, y_val)
model4.set_features(numeric, cat)
model4.fit()

0:	learn: 0.5941556	test: 0.6161616	best: 0.6161616 (0)	total: 13.8ms	remaining: 1m 8s
1:	learn: 0.5942298	test: 0.6060606	best: 0.6161616 (0)	total: 25.7ms	remaining: 1m 4s
2:	learn: 0.5946006	test: 0.6010101	best: 0.6161616 (0)	total: 36.7ms	remaining: 1m 1s
3:	learn: 0.5943781	test: 0.5959596	best: 0.6161616 (0)	total: 47.3ms	remaining: 59.1s
4:	learn: 0.5944523	test: 0.6060606	best: 0.6161616 (0)	total: 56.9ms	remaining: 56.8s
5:	learn: 0.5951939	test: 0.6161616	best: 0.6161616 (0)	total: 66.4ms	remaining: 55.3s
6:	learn: 0.5962323	test: 0.6212121	best: 0.6212121 (6)	total: 75.5ms	remaining: 53.9s
7:	learn: 0.5956390	test: 0.6212121	best: 0.6212121 (6)	total: 85.2ms	remaining: 53.2s
8:	learn: 0.5963065	test: 0.6161616	best: 0.6212121 (6)	total: 94.1ms	remaining: 52.2s
9:	learn: 0.5952681	test: 0.6111111	best: 0.6212121 (6)	total: 102ms	remaining: 51.1s
10:	learn: 0.5960098	test: 0.6111111	best: 0.6212121 (6)	total: 112ms	remaining: 50.7s
11:	learn: 0.5957873	test: 0.6111111	best: 0

In [21]:
print(f"Score without R2: {model.score(X_val, y_val)}")
print(f"Score with L2: {model2.score(X_val, y_val)}")
print(f"Score with L1: {model3.score(X_val, y_val)}")
print(f"Score wuth l2_leaf_reg: {model4.score(X_val, y_val)}")

Score without R2: 0.6262626262626263
Score with L2: 0.6161616161616161
Score with L1: 0.6313131313131313
Score wuth l2_leaf_reg: 0.6212121212121212
